# <center> Recurrent Neural Networks - Long Short Term Memory</center>

# Recurrent Neural Networks (RNN)

As it was seen beforehand Neural Networks offer a very comprehensive and efficient solution to classification problems. However could it also be applied to prediction? 

Google Translate, Apple's Siri, ... What do these applications have in common? They rely on specific Neural Networks called Recurrent Neural Networks. The core difference with what has been seen is the ability of RNNs to "remember" previous inputs, ie using sequential data. 

To explain why it is relevant to use these, one should think of an application example we will see afterwards. <br> Imagine you want to predict an Air Quality value. How do you do this? Would you rather try to start from scratch and predict it form external parameters or would you try to determine it by analyzing previous values of Air Quality? The second option seems like a fair choice, however basic Feed-Forward Neural Networks cannot remember previous values. 

Example: Air Quality Prediction in Belgium

## Limits of RNNs: The vanishing gradient problem

# Long Short Term Memory

In order to compose with RNNs short-comings, a new method was designed